In [1]:
%matplotlib notebook
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

[]

In [9]:
engine.execute('SELECT * FROM Measurement LIMIT 10').fetcall()

OperationalError: (sqlite3.OperationalError) no such table: Measurement [SQL: 'SELECT * FROM Measurement LIMIT 10'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [10]:
columns = inspector.get_columns('Measurement')
for c in columns:
    print (c['name'], c["type"])

In [11]:
columns = inspector.get_columns('Station')
for c in columns:
    print (c['name'], c["type"])

In [12]:
Station.__table__

NameError: name 'Station' is not defined

In [13]:
Measurement.__table__

NameError: name 'Measurement' is not defined

In [14]:

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

AttributeError: measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
precip_12mo = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > data_oneyear).order_by(Measurement.date).all()
print(precip_12mo)

# Calculate the date 1 year ago from today
data_oneyear=dt.date(2018, 8, 6) - dt.timedelta(days=365)
print(data_oneyear)

# Perform a query to retrieve the data and precipitation scores
precip_12mo_df=pd.DataFrame(precip_12mo)
precip_12mo_df.head()

# Save the query results as a Pandas DataFrame and set the index to the date column
mod1precip_12mo_df=precip_12mo_df.set_index('date')
mod1precip_12mo_df.head()

# Sort the dataframe by date
sortedprecip_12mo_df=precip_12mo_df.sort_values('date',ascending=True)
sortedprecip_12mo_df.head()

# Use Pandas Plotting with Matplotlib to plot the data
sortedprecip_12mo_df.plot('date','prcp',color='#E74C3C')
plt.xlabel("Date")
plt.ylabel("Precipitation (in.)")
plt.title("Precipitation Analysis (8/7/2017 - 8/6/2018)")
plt.legend(["Precipitation"])

# Define months for x-ticks labels
months = ["Aug","Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug"]

# Define x and y values
y = sortedprecip_12mo_df["prcp"].tolist()
x = np.arange(0, len(sortedprecip_12mo_df.index.tolist()), 1)
# Define months for x-axis labels
months = ["Aug","Sep", "Oct", "Nov", "Dec", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug"]

# Define x-axis labels (months) and their positioning
month_total = len(y)
month_step = int((month_total / 12)*1.03)
plt.ylim = max(y) + 1
tick_locations = [x for x in range(1, month_total, month_step)]

plt.xticks(tick_locations, months,rotation=45)
plt.tight_layout()
plt.show()

# Rotate the xticks for the dates


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
sortedprecip_12mo_df.describe()

In [ ]:
# How many stations are available in this dataset?
stations = session.query(Measurement).group_by(Measurement.station).count()
print("{} stations.".format(stations))

In [ ]:
# What are the most active stations?
busy_three=stations_des[0:3]
print("The three most active stations and their associated counts:",busy_three)

In [ ]:

# Query all tobs values
results = session.query(Measurement.tobs).all()

# Convert tuples into list
tobs_values = list(np.ravel(results))
tobs_values

In [ ]:
#Convert tobs list to data frame
tobs_values_df=pd.DataFrame([tobs_values]).T
#tobs_values_df.rename(columns={'0':'Temperature (Deg. Fah.)'}, inplace=True)
tobs_values_df.head()

In [ ]:
#Create data frame with stations and counts
stn_cnt_df=pd.DataFrame(stations_des,columns=['Station','Counts'])
stn_cnt_df.head()

In [ ]:
#Find busiest station
busiest_stn=stations_des[0][0]
busiest_stn

In [ ]:
# List the stations and the counts in descending order.
stations_des=session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()
stations_des

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
stn_temp = session.query(Measurement.station, Measurement.date, Measurement.tobs).filter(Measurement.station == busiest_stn).filter(Measurement.date > data_oneyear).order_by(Measurement.date).all()
stn_temp

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
stn_tempobs = session.query(Measurement.station, Measurement.tobs).\
    filter(Measurement.station == busiest_stn).\
    filter(Measurement.date >= data_oneyear).all()
stn_tempobs

In [ ]:
#Create data frame with stations, dates, and temperature
stn_tempobs_df=pd.DataFrame(stn_tempobs,columns=['Station','Temperature (Deg. Fah.)'])
stn_tempobs_df

In [ ]:

#Plot histogram 

hist=stn_tempobs_df.hist(bins=10, color='#2E86C1')
plt.xlabel('Temperature (Deg. Fahr.)')
plt.ylabel('Number of Observations')
plt.title('Temperature Analysis at Station (8/7/2017 - 8/6/2018)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
